In [1]:
#Imports
import numpy as np
import pandas as pd
import json
import pickle

from tqdm import tnrange, tqdm_notebook


from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


import functions 
classif = functions.classification()

In [2]:
#To load from pickle file
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}.pickle"

features = []
with open(path.format('features'), 'rb') as fr:
    features = pickle.load(fr)
    fr.close()
print(len(features))
features[0]

360


'ngram_2000_7_both_none'

In [3]:
from numpy import inf


def k_fold(features,C,cv = 10,kernel='linear'):
    """
    Returns k_fold accuracy for given C parameter
    """
    X = features.drop(columns='target')
    X=(X-X.mean())/X.std() #Normalise - REMOVE
    X.fillna(value=0,inplace=True)
    
    y = features['target']
    
    model = SVC(kernel=kernel, C=C)
    scores = cross_val_score(model, X, y, cv=cv)
    accuracy = scores.mean()
    return accuracy

#replace in autoML
def trainModel(filename):
    path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}.csv"
    row = filename.split("_")

    train = pd.read_csv(path.format("train_"+filename))
    test = pd.read_csv(path.format("test_"+filename))
    
    #Train Accuracy
    train_accuracy = k_fold(train,C=0.1,cv = 10,kernel='linear')
    
    #Test Accuracy
    X_train = train.drop(columns='target')
    #X_train.fillna(value=0,inplace=True)
    y_train = train['target']
    
    X_test= test.drop(columns='target')
    #X_test.fillna(value=0,inplace=True)
    
    y_test = test['target'] 
    
    model = SVC(kernel='linear', C=0.1).fit(X_train,y_train)
    
    predictions = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, predictions)
     
    row = row + [train_accuracy,test_accuracy]
    
    return row
f = features[0]
trainModel('ngram_2000_2_procedure_none')

['ngram',
 '2000',
 '2',
 'procedure',
 'none',
 0.7482142857142857,
 0.8225806451612904]

In [4]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/features/{}.csv"

results = []
for f in tqdm_notebook(features):
    try:
        result = trainModel(f)
        results.append(result)
    except: 
        print(f)
    
print(len(results))



720


In [5]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/results/{}.csv"

results = pd.DataFrame(results)
results.columns = ['type','dimension',"article","part","stopwords","train",'test']

results.to_csv(path.format("results_autoML"),index=False)

results.head()

,type,dimension,article,part,stopwords,train,test
0,ngram,2000,2,both,none,0.812500,0.887097
1,glove,100,2,both,none,0.736607,0.870968
2,glove,200,2,both,none,0.679464,0.887097
3,law,100,2,both,none,0.774107,0.870968
4,law,200,2,both,none,0.742857,0.919355
